
# Install fastai dependencies

In [1]:
project_name = "ears---ibb"
version = 11
#train_version = 2
#test_version = 4
repeats = 2
epochs = 50
model_name = "best_model_d_" + project_name + "_v_" + str(version) + "_r_" + str(repeats) + "_e_" + str(epochs)

In [2]:
!pip install fastai

In [3]:
from fastai.vision import *

# Download Custom Classification Data From Roboflow

In [4]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="folder", notebook="roboflow-resnet")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=folder&ref=roboflow-resnet


In [5]:
#dataset imported from Roboflow. You can sign up at roboflow.com and upload your image classification dataset
from roboflow import Roboflow
rf = Roboflow(api_key="vNXadpVSzZfevBW2gHPf")
project = rf.workspace().project(project_name)
dataset = project.version(version).download("folder")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Ears---IBB-11 in folder:: 100%|██████████| 2643/2643 [00:01<00:00, 1664.98it/s]


In [6]:
#build fastai dataset loader
np.random.seed(420)
#fastai automatically factors the ./train and ./valid folders into seperate datasets
#more details https://docs.fast.ai/vision.data.html#ImageDataLoaders.from_folder
path = Path(dataset.location)
data = ImageDataBunch.from_folder(path, valid_pct=0.2, size=224, num_workers=4).normalize(imagenet_stats)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Set up fastai Resnet model

In [8]:
from fastai.metrics import error_rate # 1 - accuracy
learn = create_cnn(data, models.resnet152, metrics=error_rate)

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:109: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [9]:
#print network layers
#learn

# Train Custom Resnet Image Classifier

In [10]:
from fastai.callbacks import *
early_stop = EarlyStoppingCallback(learn, patience=20)
save_best_model = SaveModelCallback(learn, name=model_name)

In [11]:
def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = False) -> float:
    #Run the Learning Rate Finder
    model.lr_find()
    
    #Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    min_loss_index = np.argmin(losses)
    
    
    #loss_grad = np.gradient(losses)
    lrs = model.recorder.lrs
    
    #return the learning rate that produces the minimum loss divide by 10   
    return lrs[min_loss_index] / 10

In [12]:
defaults.device = torch.device('cuda') # makes sure the gpu is used
for i in range(repeats):
  print(f'----STEP {i}----\n\n')
  #frozen training step
  learn.fit_one_cycle(epochs, callbacks=[early_stop, save_best_model])
  #load best model from frozen training
  learn.load(model_name)
  learn.unfreeze()
  optimal_lr = find_appropriate_lr(learn)
  learn.fit_one_cycle(epochs, max_lr=slice(optimal_lr/10, optimal_lr), callbacks=[early_stop, save_best_model])
  learn.load(model_name)

----STEP 0----




epoch,train_loss,valid_loss,error_rate,time
0,6.212162,5.185751,0.975410,00:36
1,5.258053,4.381114,0.930328,00:34
2,4.325746,3.479961,0.786885,00:35
3,3.393998,2.853208,0.655738,00:35
4,2.494473,2.442927,0.571721,00:36
5,1.768234,2.088693,0.461066,00:36
6,1.231991,1.854795,0.420082,00:37
7,0.872904,1.745788,0.391393,00:38
8,0.619963,1.690645,0.383197,00:39
9,0.443176,1.668720,0.393443,00:39


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Better model found at epoch 0 with valid_loss value: 5.185750961303711.
Better model found at epoch 1 with valid_loss value: 4.381113529205322.
Better model found at epoch 2 with valid_loss value: 3.4799609184265137.
Better model found at epoch 3 with valid_loss value: 2.853208065032959.
Better model found at epoch 4 with valid_loss value: 2.442927360534668.
Better model found at epoch 5 with valid_loss value: 2.0886926651000977.
Better model found at epoch 6 with valid_loss value: 1.8547946214675903.
Better model found at epoch 7 with valid_loss value: 1.7457879781723022.
Better model found at epoch 8 with valid_loss value: 1.6906450986862183.
Better model found at epoch 9 with valid_loss value: 1.6687196493148804.
Better model found at epoch 10 with valid_loss value: 1.6571882963180542.
Better model found at epoch 11 with valid_loss value: 1.5656605958938599.
Better model found at epoch 15 with valid_loss value: 1.5147500038146973.
Better model found at epoch 16 with valid_loss value

epoch,train_loss,valid_loss,error_rate,time


...

Better model found at epoch 0 with valid_loss value: 1.174485206604004.
Better model found at epoch 7 with valid_loss value: 0.9927594065666199.
Better model found at epoch 16 with valid_loss value: 0.7992968559265137.
Better model found at epoch 17 with valid_loss value: 0.680681049823761.
Better model found at epoch 18 with valid_loss value: 0.6478889584541321.
Better model found at epoch 24 with valid_loss value: 0.6414975523948669.
Better model found at epoch 25 with valid_loss value: 0.615937352180481.
Better model found at epoch 27 with valid_loss value: 0.6024968028068542.
Better model found at epoch 28 with valid_loss value: 0.49256691336631775.
Better model found at epoch 29 with valid_loss value: 0.460214227437973.
Better model found at epoch 30 with valid_loss value: 0.37506526708602905.
Better model found at epoch 31 with valid_loss value: 0.3393755853176117.
Better model found at epoch 32 with valid_loss value: 0.33666279911994934.
Better model found at epoch 37 with valid

epoch,train_loss,valid_loss,error_rate,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,error_rate,time
0,0.000947,0.306227,0.045082,00:50
1,0.000920,0.304464,0.047131,00:50
2,0.000520,0.308557,0.043033,00:50
3,0.000464,0.311644,0.047131,00:50
4,0.000362,0.304247,0.047131,00:50
5,0.000328,0.307137,0.047131,00:50
6,0.000285,0.304133,0.045082,00:50
7,0.000257,0.307753,0.047131,00:50
8,0.000232,0.305324,0.047131,00:50
9,0.000465,0.301864,0.043033,00:50


Better model found at epoch 0 with valid_loss value: 0.3062274754047394.
Better model found at epoch 1 with valid_loss value: 0.3044638931751251.
Better model found at epoch 4 with valid_loss value: 0.3042467534542084.
Better model found at epoch 6 with valid_loss value: 0.30413293838500977.
Better model found at epoch 9 with valid_loss value: 0.3018643260002136.
Better model found at epoch 14 with valid_loss value: 0.30049049854278564.
Epoch 35: early stopping


# Evaluate Classifier Performance

In [15]:
#run inference on test images
import glob
import pickle
from IPython.display import Image, display

model = learn.model
model = model.cuda()
#im_list = glob.glob(dataset_test.location + '//test/*/*.jpg')
im_list = glob.glob(dataset.location + '/test/*/*.jpg')
pred_list = []
y = []
for imageName in im_list:
    img_class = int(os.path.basename(os.path.dirname(imageName)))
    img = open_image(imageName)
    prediction = learn.predict(img)
    pred_list.append(prediction)
    y.append(img_class)
    #print(img_class)
    #print(prediction[0], prediction[1])
    #print(prediction[2])
    #s = np.argsort(prediction[2])
    #print(s[-1])
    #print(prediction[2][s[-1]])
    #print(learn.data.classes[int(s[-1])])
    #display(Image(filename=imageName))
    #print("\n")

In [16]:
def compute_rankX(pred_list, y, rank):
    count_all = 0
    count_correct = 0
    for prediction, label in zip(pred_list, y):
      s = np.argsort(prediction[2])
      internal_label = learn.data.classes.index(str(label))
      #print(s[len(s) - (rank):])
      if internal_label in s[len(s) - rank:]:
        count_correct += 1
      count_all += 1
    return count_correct / count_all * 100

In [17]:
ranks = []
print('Num of classes:', len(learn.data.classes))
for rank in range(1, len(learn.data.classes) + 1):
  acc = compute_rankX(pred_list, y, rank)
  if rank == 1:
    print('Rank 1:', acc)
  ranks.append(acc)
with open(dataset.location + '/' + model_name + '.res', 'wb') as f:
  pickle.dump(ranks, f)

from google.colab import files
files.download(dataset.location + '/' + model_name + '.res')

Num of classes: 100
Rank 1: 93.2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Save custom classification model for future use

In [19]:
from google.colab import files
files.download(dataset.location + '/models/' + model_name + '.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>